In [ ]:
import pandas as pd
import tensorflow as tf
import re
import numpy as np
from sklearn.metrics import f1_score
import transformers
import nltk
nltk.download("stopwords")

SEED = 1
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
train = pd.read_csv('../../Data/train.csv')
test = pd.read_csv('../../Data/test.csv')
test_labels = pd.read_csv('../../Data/test_labels.csv')

In [ ]:
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
N_LABELS = len(LABELS)

In [ ]:
# Preprocessing
def preprocessComments(comment):
    # Convert to lowercase, important for this bertmodel
    comment = comment.lower()

    # Remove leading and trailing spaces
    comment = comment.strip()

    # Remove consecutive spaces      
    comment = re.sub(r' +', ' ', comment)

    # Remove Newlines
    comment = re.sub(r'\n', ' ', comment)

    return comment

train.comment_text = train.comment_text.map(preprocessComments)
train_y = train[LABELS].values
test.comment_text = test.comment_text.map(preprocessComments)

if train['comment_text'].isnull().values.any():
  raise Exception("Missing data")
if test['comment_text'].isnull().values.any():
  raise Exception("Missing data")

In [ ]:
test_filtered = pd.merge(test, test_labels)
test_filtered = test_filtered.drop(test_filtered.index[test_filtered['toxic'] == -1])
comments_list = train['comment_text'].tolist()
test_comments_list = test_filtered['comment_text'].tolist()
print(test_filtered.shape)
len(test_comments_list)

In [ ]:
bert_name = "bert-base-uncased"

bert_model = transformers.TFAutoModel.from_pretrained(bert_name)

tokenizer = transformers.BertTokenizerFast.from_pretrained(
    pretrained_model_name_or_path=bert_name, 
    config=transformers.BertConfig.from_pretrained(bert_name))

TOKENS_MAX_LENGTH = 120

def create_tokenizer(comments):
  return tokenizer(
    text=comments,
    padding='longest', 
    truncation='longest_first',
    max_length=TOKENS_MAX_LENGTH,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True)
  
train_tokenizer = create_tokenizer(comments_list)
test_tokenizer = create_tokenizer(test_comments_list)

In [ ]:
att_mask = tf.keras.layers.Input(shape=(TOKENS_MAX_LENGTH,), name='attention_mask', dtype='int32') 
input_ids = tf.keras.layers.Input(shape=(TOKENS_MAX_LENGTH,), name='input_ids', dtype='int32')
input = {'attention_mask': att_mask, 'input_ids': input_ids}
x = bert_model.bert(input)

x = tf.keras.layers.GlobalAveragePooling1D()(x[0])
x = tf.keras.layers.Dense(N_LABELS, activation='sigmoid')(x)
model = tf.keras.models.Model(input, x)

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, decay=1e-5),
    metrics=['acc']
)

In [ ]:
model.fit(
    {'attention_mask': train_tokenizer['attention_mask'], 'input_ids': train_tokenizer['input_ids']},
    train_y,
    validation_split=0.2,
    epochs=1,
    batch_size=64,
    verbose=1
)

In [ ]:
# Save model weights

!mkdir -p saved_model
model.save('saved_model/bert.h5') 

In [ ]:
predictions = model.predict(
    {'attention_mask': test_tokenizer['attention_mask'], 'input_ids': test_tokenizer['input_ids']},
    batch_size=64,
    verbose=1
)

In [ ]:
avg = 0
for i, label in enumerate(LABELS):
    print(label, ":")
    pb = predictions[:, i] >= 0.5
    score = f1_score(test_filtered[label], pb)
    print(score)
    avg += score

avg /= N_LABELS
print("Average f1-score:", avg)